Resources Used
- wget.download('https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/_downloads/da4babe668a8afb093cc7776d7e630f3/generate_tfrecord.py')
- Setup https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/install.html

# 0. Setup Paths

In [1]:
WORKSPACE_PATH = 'Tensorflow/workspace'
SCRIPTS_PATH = 'Tensorflow/scripts'
APIMODEL_PATH = 'Tensorflow/models'
ANNOTATION_PATH = WORKSPACE_PATH+'/annotations'
IMAGE_PATH = WORKSPACE_PATH+'/images'
MODEL_PATH = WORKSPACE_PATH+'/models'
PRETRAINED_MODEL_PATH = WORKSPACE_PATH+'/pre-trained-models'
CONFIG_PATH = MODEL_PATH+'/my_ssd_mobnet/pipeline.config'
CHECKPOINT_PATH = MODEL_PATH+'/my_ssd_mobnet/'
# create pipeline.config Tensorflow/models/my_ssd_mobnet/pipeline.config paste '
# delete frow 172 fine_tune_checkpoint_version: V2

# 1. Create Label Map

In [2]:
#adding the labels for each word we are creating
labels = [{'name':'book', 'id':1}, 
          {'name':'doctor', 'id':2},
         {'name':'down', 'id':3},
          {'name':'drink', 'id':4},
          {'name':'eat', 'id':5},
          {'name':'fork', 'id':6},
          {'name':'go', 'id':7},
          {'name':'great', 'id':8},
          {'name':'hello', 'id':9},
          {'name':'help', 'id':10},
          {'name':'i_love_you', 'id':11},
          {'name':'more', 'id':12},
          {'name':'name', 'id':13},
          {'name':'no', 'id':14},
          {'name':'police', 'id':15},
          {'name':'quiet', 'id':16},
          {'name':'stop', 'id':17},
          {'name':'up', 'id':18},
          {'name':'world', 'id':19},
          {'name':'yes', 'id':20},
         ]

#the label map for above labels for TenselFlow Library detection
with open(ANNOTATION_PATH + '\label_map.pbtxt', 'w') as f:  #create folder for records at label.map in annotation file
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

# 2. Create TensorFlow records

In [3]:
# Ensures data is in the correct format. exuecutes cammand inside our notebook
# https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/install.html
# load $ pip install tensorflow-object-detection-api
# /generate_tfrecord.py allows us to pass variables from notebook to command line
# training images and test images 
!python {SCRIPTS_PATH + '/generate_tfrecord.py'} -x {IMAGE_PATH + '/train'} -l {ANNOTATION_PATH + '/label_map.pbtxt'} -o {ANNOTATION_PATH + '/train.record'}
!python {SCRIPTS_PATH + '/generate_tfrecord.py'} -x{IMAGE_PATH + '/test'} -l {ANNOTATION_PATH + '/label_map.pbtxt'} -o {ANNOTATION_PATH + '/test.record'}

Successfully created the TFRecord file: Tensorflow/workspace/annotations/train.record
Successfully created the TFRecord file: Tensorflow/workspace/annotations/test.record


# 3. Download TF Models Pretrained Models from Tensorflow Model Zoo

In [4]:
# I think this only needs to be done once, clowns down files from githup to train the model
# !cd Tensorflow && git clone https://github.com/tensorflow/models

In [5]:

# this is code to download a different starting model from TensorFlow 2 Detection Model Zoo 
# wget.download('http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz')
# !mv ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz {PRETRAINED_MODEL_PATH}
# !cd {PRETRAINED_MODEL_PATH} && tar -zxvf ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz

# 4. Copy Model Config to Training Folder

In [6]:
# create new folder
CUSTOM_MODEL_NAME = 'my_ssd_mobnet' 

In [7]:
# pre-trained model leveraging that, uses pretrained paths from above, 

# !mkdir {'Tensorflow\workspace\models\\'+CUSTOM_MODEL_NAME}
# !cp {PRETRAINED_MODEL_PATH+'/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config'} {MODEL_PATH+'/'+CUSTOM_MODEL_NAME}

# 5. Update Config For Transfer Learning

In [8]:
import tensorflow as tf
from object_detection.utils import config_util # pip install tensorflow-object-detection-api
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [9]:
CONFIG_PATH = MODEL_PATH+'/'+CUSTOM_MODEL_NAME+'/pipeline.config'
config = config_util.get_configs_from_pipeline_file(CONFIG_PATH)

In [10]:
config

{'model': ssd {
   num_classes: 20
   image_resizer {
     fixed_shape_resizer {
       height: 320
       width: 320
     }
   }
   feature_extractor {
     type: "ssd_mobilenet_v2_fpn_keras"
     depth_multiplier: 1.0
     min_depth: 16
     conv_hyperparams {
       regularizer {
         l2_regularizer {
           weight: 3.9999998989515007e-05
         }
       }
       initializer {
         random_normal_initializer {
           mean: 0.0
           stddev: 0.009999999776482582
         }
       }
       activation: RELU_6
       batch_norm {
         decay: 0.996999979019165
         scale: true
         epsilon: 0.0010000000474974513
       }
     }
     use_depthwise: true
     override_base_feature_extractor_hyperparams: true
     fpn {
       min_level: 3
       max_level: 7
       additional_layer_depth: 128
     }
   }
   box_coder {
     faster_rcnn_box_coder {
       y_scale: 10.0
       x_scale: 10.0
       height_scale: 5.0
       width_scale: 5.0
     }
   }
   matc

In [11]:
# config_text = text_format.MessageToString(pipeline_config)
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(CONFIG_PATH, "r") as f:                                                                                                                                                                                                                     
    proto_str = f.read()                                                                                                                                                                                                                                          
    text_format.Merge(proto_str, pipeline_config)  

In [12]:
pipeline_config.model.ssd.num_classes = 20  # !!!!! need to update this to number of words (objects)
pipeline_config.train_config.batch_size = 4  #  can be higher if bigger GPU
pipeline_config.train_config.fine_tune_checkpoint = PRETRAINED_MODEL_PATH+'/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0'
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path= ANNOTATION_PATH + '/label_map.pbtxt'
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [ANNOTATION_PATH + '/train.record']
pipeline_config.eval_input_reader[0].label_map_path = ANNOTATION_PATH + '/label_map.pbtxt'
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [ANNOTATION_PATH + '/test.record']

In [13]:
config_text = text_format.MessageToString(pipeline_config)                                                                                                                                                                                                        
with tf.io.gfile.GFile(CONFIG_PATH, "wb") as f:                                                                                                                                                                                                                     
    f.write(config_text)   

# 6. Train the model

In [14]:
# step 1 change this from 5,000 to 10,000 to 20,000
# step 2 copy this into cd RealTimeObjectDirection and train model into command prompt
# look for loss 0.099

print("""python {}/research/object_detection/model_main_tf2.py --model_dir={}/{} --pipeline_config_path={}/{}/pipeline.config --num_train_steps=10000""".format(APIMODEL_PATH, MODEL_PATH,CUSTOM_MODEL_NAME,MODEL_PATH,CUSTOM_MODEL_NAME))

python Tensorflow/models/research/object_detection/model_main_tf2.py --model_dir=Tensorflow/workspace/models/my_ssd_mobnet --pipeline_config_path=Tensorflow/workspace/models/my_ssd_mobnet/pipeline.config --num_train_steps=10000


# 7. Load Train Model From Checkpoint

In [17]:
#restarted kernel
import os
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

In [18]:
# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(CONFIG_PATH)
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(CHECKPOINT_PATH, 'ckpt-1')).expect_partial()  # ckpt is from Tensorflow/workplace/models/my_ssd... folder

@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    print(i)
    return detections

In [19]:
i = 0

# 8. Detect in Real-Time

In [20]:
import cv2 
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

In [21]:
category_index = label_map_util.create_category_index_from_labelmap(ANNOTATION_PATH+'/label_map.pbtxt')
category_index

{1: {'id': 1, 'name': 'book'},
 2: {'id': 2, 'name': 'doctor'},
 3: {'id': 3, 'name': 'down'},
 4: {'id': 4, 'name': 'drink'},
 5: {'id': 5, 'name': 'eat'},
 6: {'id': 6, 'name': 'fork'},
 7: {'id': 7, 'name': 'go'},
 8: {'id': 8, 'name': 'great'},
 9: {'id': 9, 'name': 'hello'},
 10: {'id': 10, 'name': 'help'},
 11: {'id': 11, 'name': 'i_love_you'},
 12: {'id': 12, 'name': 'more'},
 13: {'id': 13, 'name': 'name'},
 14: {'id': 14, 'name': 'no'},
 15: {'id': 15, 'name': 'police'},
 16: {'id': 16, 'name': 'quiet'},
 17: {'id': 17, 'name': 'stop'},
 18: {'id': 18, 'name': 'up'},
 19: {'id': 19, 'name': 'world'},
 20: {'id': 20, 'name': 'yes'}}

In [22]:
# cap.release()   # this can be deleted

In [23]:
# Setup capture
cap = cv2.VideoCapture(0)   # access webcam
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))  
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
print(width)
print(height)

640
480


In [24]:


while (True): 
    ret, frame = cap.read()  # get image from camera
    image_np = np.array(frame)  # convert to numpy Array
    
    # pass in np.expand_dims to place in another array
    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32) # convert to tensor flwo tensor
    detections = detect_fn(input_tensor)  # make the detection from function above in step 7
    
    num_detections = int(detections.pop('num_detections'))  # the number of detections
    
    detections = {key: value[0, :num_detections].numpy()     #preprocessing
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64) # collect the dectection classes

    label_id_offset = 1   # start at 1 and not 0 because our category index (list of words) starts at 1
    image_np_with_detections = image_np.copy()  # make a copy as backup of raw image from webcam
    print('test1', i)
    viz_utils.visualize_boxes_and_labels_on_image_array(   # transform array, place score and box on image
                image_np_with_detections,
                detections['detection_boxes'],
                detections['detection_classes']+label_id_offset,
                detections['detection_scores'],
                category_index,
                use_normalized_coordinates=True,  # position box correctly
                max_boxes_to_draw=1,   # number of boxes to draw
                min_score_thresh=0.3,   # miminum score threshold
                agnostic_mode=False) # not to differ between post and neg

#     img = cv2.imread('\Tensorflow\workspace\images\test\hello.d06b8f38-6e15-11eb-9de2-0c7a15e4857b.jpg',0)
#     cv2.imshow('image', img)
    
  #This problem says that an assertion is failed. If the file doesn't exist in the given path,cv2 returns this error. So check if the file exists in your given path.  
#Creating a new environment and installing opencv-python by pip worked
#     plt.imshow(image_np_with_detections) #,  cv2.resize(image_np_with_detections, (800, 600)))
    cv2.imshow('object detection',  cv2.resize(image_np_with_detections, (800, 600)))  # display transformed array back onto screen
#     cv2.imshow('object detection', cv2.resize(image_np, (800,600)))
   
    if cv2.waitKey(1) & 0xFF == ord('q'):  # break out of loop
        cap.release()
        break
# https://stackoverflow.com/questions/28776053/opencv-gtk2-x-error
# for windows just uninstall the OpenCV pip uninstall opencv-python and reinstall pip install opencv-python        
        
        
# sudo apt-get purge python-opencv
# sudo apt-get update
# sudo apt-get install python-opencv

0
0
test1 0


error: OpenCV(4.5.1) C:\Users\appveyor\AppData\Local\Temp\1\pip-req-build-cl8wq7nq\opencv\modules\highgui\src\window.cpp:651: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvShowImage'


In [ ]:
sudo apt-get purge python-opencv
sudo apt-get update
sudo apt-get install python-opencv

In [ ]:
cv2.__version__
# cv.__version__

In [ ]:

cv2.imshow('object detection',  cv2.resize(image_np_with_detections, (800, 600)))  # display transformed array back onto screen


In [26]:
cap.release() 

In [ ]:
cv2.destroyAllWindows()

In [ ]:
# suppose this was the source 
cap = cv2.VideoCapture('data/input.mpg')
# Get width and height of the frame of video
width  = cap.get(3) # float width
height = cap.get(4) # float height    
# Make a video writer to see if video being taken as input inflict any changes you make
fourcc = cv2.VideoWriter_fourcc(*"MJPG")
out_video = cv2.VideoWriter('result/output.avi', fourcc, 20.0, (int(width), int(height)), True)
# Then try this
while(cap.isOpened()):
        # Read each frame where ret is a return boollean value(True or False)
        ret, frame = cap.read()
        # if return is true continue because if it isn't then frame is of NoneType in that case you cant work on that frame
        if ret:
            # Any preprocessing you want to make on the frame goes here
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            # See if preprocessing is working on the frame it should
            cv2.imshow('frame', gray)
            # finally write your preprocessed frame into your output video
            out_video.write(gray) # write the modifies frame into output video 
            # to forcefully stop the running loop and break out, if it doesnt work use ctlr+c
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
       # break out if frame has return NoneType this means that once script encounters such frame it breaks out 
       # You will get the error otherwise 
        else:
            break
# this will tell you from which frame the video is corrupted and need to be changed 
# this could be due to an empty frame. Empty frame might be a result of mistake made while creating or editing video in tools #like adobe premier pro or other products alike 
# You can check you output video it will contains all frame before the error is encountered

In [ ]:
import numpy as np
import cv2


img = cv2.imread('messi5.jpg',0)
cv2.imshow('image',img)
k = cv2.waitKey(0)
if k == 27:         # wait for ESC key to exit
    cv2.destroyAllWindows()
elif k == ord('s'): # wait for 's' key to save and exit
    cv2.imwrite('messigray.png',img)
    cv2.destroyAllWindows()
# same error but in imread   https://stackoverflow.com/questions/50783177/opencv-the-function-is-not-implemented-rebuild-the-library-with-windows
# 7:39 am pip install opencv-python   
# pip install opencv-contrib-python

# get CV2 info:  python -c "import cv2; print(cv2.getBuildInformation())"

In [ ]:
https://www.youtube.com/watch?v=IOI0o3Cxv9Q

Hi, sir. Me again from the yesterday instllation video. 
I have gone through this video and i have train my model. 
But there is an error in the final step which is  cv2.imshow('object detection',  cv2.resize(image_np_with_detections, (800, 600)))
and it gives the error like this
error: OpenCV(4.5.1) C:\Users\appveyor\AppData\Local\Temp\1\pip-req-build-memyuvq3\opencv\modules\highgui\src\window.cpp:651: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvShowImage'

I think i need to install opencv but how to install and which drive or location i supposed to install it?
Hi, sir. Me again from the yesterday instllation video. 
I have gone through this video and i have train my model. 
But there is an error in the final step which is  cv2.imshow('object detection',  cv2.resize(image_np_with_detections, (800, 600)))
and it gives the error like this
error: OpenCV(4.5.1) C:\Users\appveyor\AppData\Local\Temp\1\pip-req-build-memyuvq3\opencv\modules\highgui\src\window.cpp:651: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvShowImage'

I think i need to install opencv but how to install and which drive or location i supposed to install it?

JiaJun Loh
2 weeks ago (edited)
Hi sir, i finally solved the problem with pip install --user opencv-contrib-python. 





In [ ]:
To anyone in the future: I had to put opencv_ffmpeg310.dll into my working directory. Otherwise the VideoCapture basically just failed silently. – MrZander Jan 28 '19 at 23:21